# Kernel SHAP vs Tree SHAP
Experiments to understand the time complexity of SHAP approximations

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import sklearn.datasets as ds

import datetime

import shap
shap.initjs()

In [2]:
# Functions
def runSHAP(n,kernel=True): 
    """
    Calculate shap values and return time taken
        n: number of SHAP values to calculate
        kernel: set False if using TreeSHAP 
    """
    
    x_sample = X[np.random.choice(X.shape[0], n, replace=True)]
    
    begin = datetime.datetime.now()
    if kernel:
        #Caculate SHAP values using KernelSHAP
        shap_values = kernelSHAP.shap_values(x_sample,l1_reg=False)
        time = datetime.datetime.now() - begin
        print("Kernel {}: ".format(n), time)
    else:
        #Caculate SHAP values using TreeSHAP
        shap_values = treeSHAP(x_sample)
        time = datetime.datetime.now() - begin
        print("Tree {}: ".format(n), time)
    
    return time

def model_properties(model):
    """Returns average depth and number of features and leaves of a random forest"""
    
    depths = []
    features = []
    leaves = []
    
    for tree in model.estimators_:
        depths.append(tree.get_depth())
        leaves.append(tree.get_n_leaves())
        n_feat = len(set(tree.tree_.feature)) -1 
        features.append(n_feat)
        
    return np.mean(depths), np.mean(features), np.mean(leaves)

## Experiment 1: Number of samples

In [3]:
#Simulate regression data
data = ds.make_regression(n_samples=10000, n_features=10, n_informative=8, n_targets=1)

y= data[1]
X = data[0]

feature_names = range(len(X))

In [ ]:
#Train model
model = RandomForestRegressor(n_estimators=100,max_depth=4,random_state=0)
model.fit(X, y)

In [5]:
#Get shap estimators
kernelSHAP = shap.KernelExplainer(model.predict,shap.sample(X, 10))
treeSHAP = shap.TreeExplainer(model)

In [ ]:
results = []
for n in [10,100,1000,2000,5000,10000]*3:
    #Calculate SHAP Values
    kernel_time = runSHAP(n=n)
    tree_time = runSHAP(n=n,kernel=False)
    
    result = [n,kernel_time,tree_time]
    results.append(result)
    
results_1 = pd.DataFrame(results,columns = ['n','kernelSHAP','treeSHAP'])

In [ ]:
avg_1 = results_1.groupby(by='n',as_index=False).mean()
avg_1

In [ ]:
k_sec

In [ ]:
#Find average run time
avg_1 = results_1.groupby(by='n',as_index=False).mean()

k_sec = [t.total_seconds() for t in avg_1['kernelSHAP']]
t_sec = [t.total_seconds() for t in avg_1['treeSHAP']]
n = avg_1['n']

#Proportional run time
print((k_sec/n)/(t_sec/n))

#Plot run time by number of observations
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,6))

plt.plot(n, k_sec, linestyle='-', linewidth=2,marker='o',label = 'KernelSHAP')
plt.plot(n, t_sec, linestyle='-', linewidth=2,marker='o',label = 'TreeSHAP')

plt.ylabel('Time (seconds)',size=20)
plt.xlabel('Number of observations',size=20)
plt.legend(fontsize=15)

In [ ]:
#Number of observations
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,6))

plt.plot(n, t_sec, linestyle='-', color='#F87F0E',linewidth=2,marker='o',label = 'TreeSHAP')

plt.ylabel('Time (seconds)',size=20)
plt.xlabel('Number of observations',size=20)
plt.legend(fontsize=15)

## Experiment 2: number of features
    

In [ ]:
results = []

for n_features, n_informative in zip([2,4,6,8,10,12,13,14,16,18,20]*3,[2,4,6,8,10,12,13,14,16,18,20]*3):
    
    #Simulate regression data
    data = ds.make_regression(n_samples=10000, n_features=n_features, n_informative=n_informative, n_targets=1,noise=0.1)

    y= data[1]
    X = data[0]

    feature_names = range(len(X))

    #Train model
    model = RandomForestRegressor(n_estimators=100,max_depth=10,random_state=0)
    model.fit(X, y)
    
    #get model properties
    avg_depth, avg_feat, avg_leaves = model_properties(model)
    
    #Get shap estimators
    kernelSHAP = shap.KernelExplainer(model.predict,shap.sample(X, 10))
    treeSHAP = shap.TreeExplainer(model)
    
    #Calculate SHAP values
    kernel_time = runSHAP(n=100)
    tree_time = runSHAP(n=100,kernel=False)
    
    result = [n_features, avg_depth, avg_feat, avg_leaves, kernel_time,tree_time]
    results.append(result)

results_2 = pd.DataFrame(results,columns = ['n_features','avg_depth', 'avg_feat', 'avg_leaves','kernelSHAP','treeSHAP'])


In [ ]:
#Get average run time
avg_2 = results_2[['n_features','kernelSHAP','treeSHAP']].groupby(by='n_features',as_index=False).mean()

k_sec = [t.total_seconds() for t in avg_2['kernelSHAP']]
t_sec = [t.total_seconds() for t in avg_2['treeSHAP']]
n = avg_2['n_features']

print((k_sec/n)/(t_sec/n))

#Plot run time by number of features
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,6))

plt.plot(n, k_sec, linestyle='-', linewidth=2,marker='o',label = 'KernelSHAP')
plt.plot(n, t_sec, linestyle='-', linewidth=2,marker='o',label = 'TreeSHAP')

plt.ylabel('Time (seconds)',size=20)
plt.xlabel('Number of features',size=20)
plt.legend(fontsize=15)

## Experiment 3: number of trees

In [ ]:
#Simulate regression data
data = ds.make_regression(n_samples=10000, n_features=10, n_informative=8, n_targets=1)

y= data[1]
X = data[0]

feature_names = range(len(X))

In [ ]:
results = []

for trees in [10,20,50,100,200,500,1000]*3:
    #Train model
    model = RandomForestRegressor(n_estimators=trees,max_depth=4,random_state=0)
    model.fit(X, y)
    
    #Get shap estimators
    kernelSHAP = shap.KernelExplainer(model.predict,shap.sample(X, 10))
    treeSHAP = shap.TreeExplainer(model)
    
    #Calculate SHAP Values
    kernel_time = runSHAP(n=100)
    tree_time = runSHAP(n=100,kernel=False)
    
    result = [trees,kernel_time,tree_time]
    results.append(result)

results_3 = pd.DataFrame(results,columns = ['trees','kernelSHAP','treeSHAP'])

In [ ]:
#Get average run time
avg_3 = results_3.groupby(by='trees',as_index=False).mean()

k_sec = [t.total_seconds() for t in avg_3['kernelSHAP']]
t_sec = [t.total_seconds() for t in avg_3['treeSHAP']]
trees = avg_3['trees']

print((k_sec/trees)/(t_sec/trees))

#Plot run time by number of trees
fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(20,10))

ax[0].plot(trees, k_sec, linestyle='-', linewidth=2,marker='o',label = 'KernelSHAP')
ax[0].set_ylabel('Time (seconds)',size=20)
ax[0].set_xlabel('Number of trees',size=20)
ax[0].legend(fontsize=15)

ax[1].plot(trees, t_sec, color='#F87F0E', linewidth=2,marker='o',label = 'TreeSHAP')
ax[1].set_ylabel('Time (seconds)',size=20)
ax[1].set_xlabel('Number of trees',size=20)
ax[1].legend(fontsize=15)

## Experiment 4: tree depth

In [ ]:
#Simulate regression data
data = ds.make_regression(n_samples=10000, n_features=10, n_informative=8, n_targets=1)

y= data[1]
X = data[0]

feature_names = range(len(X))

results = []

#for depth in [2,4,6]:
for depth in [2,4,6,8,10,15,20]*3:

    #Train model
    model = RandomForestRegressor(n_estimators=100,max_depth=depth,random_state=0)
    model.fit(X, y)
    
    #get model properties
    avg_depth, avg_feat, avg_leaves = model_properties(model)
    
    #Get shap estimators
    kernelSHAP = shap.KernelExplainer(model.predict,shap.sample(X, 10))
    treeSHAP = shap.TreeExplainer(model)
    
    #Calculate SHAP values
    kernel_time = runSHAP(n=100)
    tree_time = runSHAP(n=100,kernel=False)
    
    result = [depth, avg_depth, avg_feat, avg_leaves, kernel_time,tree_time]
    results.append(result)

results_4 = pd.DataFrame(results,columns = ['depth','avg_depth', 'avg_feat', 'avg_leaves','kernelSHAP','treeSHAP'])

In [ ]:
#Get average run time
avg_4 = results_4[['depth','kernelSHAP','treeSHAP']].groupby(by='depth',as_index=False).mean()

k_sec = [t.total_seconds() for t in avg_4['kernelSHAP']]
t_sec = [t.total_seconds() for t in avg_4['treeSHAP']]
depth = avg_4['depth']

#Plot run tume by tree depth
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,6))

plt.plot(depth, k_sec, linestyle='-', linewidth=2,marker='o',label = 'KernelSHAP')
plt.plot(depth, t_sec, linestyle='-', linewidth=2,marker='o',label = 'TreeSHAP')
plt.legend(fontsize=15)

plt.ylabel('Time (seconds)',size=20)
plt.xlabel('Tree depth',size=20)

In [ ]:
#Other factors
r4 = results_4[['depth','avg_depth','avg_feat','avg_leaves']].groupby(by='depth',as_index=False).mean()

fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(20,10))

ax[0].plot(r4['depth'], r4['avg_feat'], linestyle='-', linewidth=2,marker='o')
ax[0].set_ylabel('Average features',size=20)
ax[0].set_xlabel('Tree depth',size=20)

ax[1].plot(r4['depth'], r4['avg_leaves'], color='#F87F0E', linewidth=2,marker='o')
ax[1].set_ylabel('Average leaves',size=20)
ax[1].set_xlabel('Tree depth',size=20)

# Archive 

In [ ]:
#
data = ds.make_regression(n_samples=10000, n_features=10, n_informative=8, n_targets=1)

y= data[1]
X = data[0]

feature_names = range(len(X))

depth = 10 # vary this value 
model = RandomForestRegressor(n_estimators=100,max_depth=depth,random_state=0)
model.fit(X, y)

model_properties(model)

In [ ]:
#Simulate regression data
data = ds.make_regression(n_samples=10000, n_features=20, n_informative=20, n_targets=1,noise=0.1)

y= data[1]
X = data[0]

feature_names = range(len(X))

#Train model
model = RandomForestRegressor(n_estimators=100,max_depth=10,random_state=0)
model.fit(X, y)

#get model properties
avg_depth, avg_feat, avg_leaves = model_properties(model)


#Get shap estimators
treeSHAP = shap.TreeExplainer(model)
kernelSHAP = shap.KernelExplainer(model.predict,shap.sample(X, 20))

#get shap values 
x_sample = X[np.random.choice(X.shape[0], 100, replace=True)]
sv_tree = treeSHAP.shap_values(x_sample)
sv_kernel = kernelSHAP.shap_values(x_sample,l1_reg=0.1)

print(len(sv_tree[0]),len(sv_kernel[0]))